# StackAPI

#### Import the necessary libraries here:

In [30]:
# your code here
from stackapi import StackAPI
from pandas.io.json import json_normalize
import datetime
import time
import requests
import pandas as pd
import json

#### Question 1: Find the questions and answers of last month.

In [31]:
##Import StackAPI
SITE = StackAPI('stackoverflow')

In [36]:
## Create function to calculate timestamp
def create_timestamp(date_time):
    date = datetime.datetime.strptime(date_time, '%d/%m/%Y %H:%M:%S')
    return int(time.mktime(date.timetuple()))

fromdate = create_timestamp('29/12/2019 00:00:00')
todate = create_timestamp('29/01/2020 00:00:00')

In [37]:
##Request questions from last month and store it in a variable
questions = SITE.fetch('questions', fromdate=fromdate, todate=todate)

##Create dataframe with the questions list
df_questions_items = json_normalize(questions ['items'])
table_questions = df_questions_items[[ 'title', 'question_id' ]]
table_questions.head(10)

,title,question_id
0,How to parallelize requests with MySQL+PHP+nginx?,59791225
1,How to control all file extensions or not anot...,59912963
2,Proper way of testing whether a Docker Compose...,59856896
3,SilverStripe not saving page content,59911265
4,How to hide the default console output in Robo...,59912100
5,Open whatsapp//: in webview,59524579
6,Additional search filter text box?,59902481
7,Slack Notifications in Laravel - to() function...,59586307
8,Selecting a newly added row in jqGrid,59865676
9,Mongoose can&#39;t find recent saved,59907547


In [18]:
##Request answers from last month and store it in a variable
answers = SITE.fetch('answers', fromdate=fromdate, todate=todate)

##Create dataframe with the answers list
df_answers_items = pd.DataFrame(answers ['items'])
table_answers = df_answers_items[[ 'question_id', 'answer_id' ]]
table_answers

,question_id,answer_id
0,16576499,59791310
1,59512559,59832244
2,59794277,59795099
3,59273338,59552593
4,59954995,59956655
...,...,...
495,20152479,59753370
496,59957957,59958212
497,59940525,59945761
498,41980576,59953672


In [17]:
##Merge both tables
total_list = pd.merge(table_questions, table_answers, on='question_id', how='outer')
total_list.head(10)

,title,question_id,answer_id
0,Pset7 - Movies Stuck on 12 and 13 SQL?,59706969,NaN
1,Interrogate the C compiler about data types wh...,59802368,NaN
2,Swift: Precondition error on device but not em...,59914875,NaN
3,"Spring MVC App, in form:input path= converts g...",59918598,NaN
4,JDBI: How to map query as list of objects,59777290,NaN
5,Mac iCloud alters Xcode Pods breaking my build,59739295,59927270.0
6,Is using LAMP as a server component for my gam...,59925149,NaN
7,Spring JPA with Postgres ltree extension,59956476,NaN
8,Generating random numbers from custom continuo...,59939092,59951729.0
9,Python shared lock between Threads and EventLoops,59950212,NaN


In [22]:
## The reason why there are so many NaN on the answer column, even though some of those answers have indeed questions, is that
## the call used above returns a limited number of elements. To retrieve the whole list of elements, a function to retrieve
## all the pages needs to be put in place. However, since the amount of elements is very high, the query is very consuming.

## As an example, the code below shows how the information could be retieved. The call has been limited to 3 pages.

page = 1
result = []
more = True
while more == True:
    questions = SITE.fetch('questions', fromdate=1577577599, todate=1580255999, page=page)
    result.append(questions)
    more = questions['has_more']
    page +=1
    if page > 3:
        break
        
questions = pd.DataFrame()
for i in result:
    df_results = json_normalize(i['items'])
    questions = questions.append(df_results)
    
questions[[ 'title', 'question_id' ]]


,tags,is_answered,view_count,bounty_amount,bounty_closes_date,answer_count,score,last_activity_date,creation_date,last_edit_date,...,owner.reputation,owner.user_id,owner.user_type,owner.profile_image,owner.display_name,owner.link,accepted_answer_id,owner.accept_rate,closed_date,closed_reason
0,"[php, .htaccess]",False,74,50.0,1.581231e+09,3,1,1580738649,1580211602,1.580626e+09,...,449,10127393,registered,https://www.gravatar.com/avatar/e7e0e571c37bff...,glittergirl,https://stackoverflow.com/users/10127393/glitt...,NaN,NaN,NaN,NaN
1,"[c++, visual-studio, cmake]",True,38,NaN,NaN,1,1,1580738468,1579271701,1.580738e+09,...,48,6346852,registered,https://www.gravatar.com/avatar/ad29693718e93b...,Necktschnagge,https://stackoverflow.com/users/6346852/neckts...,59789571.0,NaN,NaN,NaN
2,"[java, maven, jenkins, maven-central]",True,21148,NaN,NaN,19,89,1580738404,1579152712,1.579274e+09,...,1182,6082483,registered,https://www.gravatar.com/avatar/ee745bd615bab1...,Arunan Sugunakumar,https://stackoverflow.com/users/6082483/arunan...,59764670.0,NaN,NaN,NaN
3,"[python, datetime, time, websocket, timedelta]",False,18,NaN,NaN,1,0,1580738372,1579132797,NaN,...,9,12280166,registered,https://www.gravatar.com/avatar/4687e40c063ddb...,rctrading,https://stackoverflow.com/users/12280166/rctra...,NaN,NaN,NaN,NaN
4,"[android, angular, ionic4, google-places-api]",False,77,NaN,NaN,2,2,1580738313,1579795746,1.580244e+09,...,4058,2469747,registered,https://i.stack.imgur.com/rCpFy.jpg?s=128&g=1,Unferth,https://stackoverflow.com/users/2469747/unferth,NaN,60.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,"[python, linux, windows, usb, pyusb]",False,18,NaN,NaN,1,1,1580638338,1579944318,NaN,...,11,3166273,registered,https://www.gravatar.com/avatar/efb42f3e5e28e1...,user3166273,https://stackoverflow.com/users/3166273/user31...,NaN,NaN,NaN,NaN
496,"[python, python-3.x, web-scraping]",True,32,NaN,NaN,1,3,1580638332,1579828369,1.580638e+09,...,45,12762898,registered,https://www.gravatar.com/avatar/3e021a55de7efb...,wzl,https://stackoverflow.com/users/12762898/wzl,59889231.0,NaN,NaN,NaN
497,"[makefile, cmake]",True,34,NaN,NaN,2,0,1580637507,1578521103,NaN,...,33,8721857,registered,https://www.gravatar.com/avatar/536f3d48fe6e07...,Potomane555,https://stackoverflow.com/users/8721857/potoma...,60024973.0,NaN,NaN,NaN
498,"[reactjs, material-ui]",False,8,NaN,NaN,0,0,1580637098,1580152730,1.580637e+09,...,2167,1734815,registered,https://i.stack.imgur.com/pB5aj.png?s=128&g=1,Danielo515,https://stackoverflow.com/users/1734815/daniel...,NaN,56.0,NaN,NaN


#### Question 2: Find the most voted question today with at least a score of 5 and tagged with 'python'. 

In [38]:
# your code here
##Request questions from today tagged 'python' and with a min score of 5 and store it in a variable

fromdate2 = create_timestamp('29/01/2020 00:00:00')
todate2 = create_timestamp('29/01/2020 23:59:59')

most_voted_question_over_5 = SITE.fetch('questions', fromdate=fromdate2, todate=todate2, min=5, sort='votes', tagged='python')
most_voted_question_over_5 

{'backoff': 0,
 'has_more': False,
 'page': 1,
 'quota_max': 300,
 'quota_remaining': 300,
 'total': 0,
 'items': []}

In [40]:
##There are no questions with a score above 5 today. To prove it, we launch the query without the score filter and sort the
##received data by score.

##Request questions from today tagged 'python' and store it in a variable
most_voted_question = SITE.fetch('questions', fromdate=fromdate2, todate=todate2, sort='votes', tagged='python')

##Create dataframe with list of questions and order by score
df_most_voted_question = json_normalize(most_voted_question['items'])
table_most_voted_question = df_most_voted_question[[ 'title', 'creation_date', 'score', 'tags' ]]
table_most_voted_question.sort_values(by=['score'], ascending=False)

,title,creation_date,score,tags
0,Python : Functional code speed is faster than ...,1580287061,4,"[python, heap]"
2,How to make QTextTable to cover whole document...,1580261387,4,"[python, python-3.x, pyqt, pyqt5, qtextdocument]"
1,Common or pythonic __repr__ methods,1580262553,4,"[python, python-3.x, repr]"
7,"How to add static(html, css, js) files in pyin...",1580297350,3,"[python, pyqt, pyqt5, pyinstaller, qwebenginev..."
11,Remove redundant points in a line,1580261748,3,"[python, arrays, list, pygame]"
...,...,...,...,...
308,Handling imports in externally-called multi-fi...,1580317569,0,[python]
307,How to use list() when in an ipdb session?,1580317843,0,"[python, python-3.x, ipdb]"
306,How to update a GUI window in Python?,1580319422,0,"[python, twisted, pysimplegui, alphavantage, a..."
305,How to use django-meta module(or any easier wa...,1580319494,0,"[python, django, django-templates]"


#### Question 3: Find the answers with id 6784 and 6473.

In [41]:
# your code here
id_answers = SITE.fetch('answers', ids=[6784, 6473])

In [42]:
df_id_answers = json_normalize(id_answers['items'])
df_id_answers = pd.DataFrame(df_id_answers.loc[:, 'is_accepted': 'owner.user_id'])
df_id_answers

,is_accepted,score,last_activity_date,creation_date,answer_id,question_id,owner.reputation,owner.user_id
0,False,0,1218291583,1218291583,6784,6778,28539,758


In [43]:
##Only one answer is returned. The query for answer id 6473 returns empty
id_answers_2 = SITE.fetch('answers', ids=[6473])
print(id_answers_2)

{'backoff': 0, 'has_more': False, 'page': 1, 'quota_max': 300, 'quota_remaining': 300, 'total': 0, 'items': []}
